In [ ]:
import seaborn.objects as so
import json
import pandas as pd
import glob
from pandas import DataFrame
import re
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
# Load results

results_folder = "../results"

# results/<tag>/<timestamp>/<size>.json
file_patter = re.compile(f"{results_folder}/([^/]+)/([^/]+)/([^/]+).json")
file_glob = f"{results_folder}/*/*/*.json"


def json_into_df(file_path: str) -> DataFrame:
    match = file_patter.fullmatch(file_path)
    if match is None:
        raise

    with open(file_path) as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    df["tag"] = match.group(1)
    df["timestamp"] = match.group(2)
    df["size"] = int(match.group(3))
    df["tag/timestamp"] = df["tag"] + '/' + df["timestamp"]
    return df


def get_all_results():
    jsons = glob.glob(file_glob)
    dfs = map(json_into_df, jsons)
    return pd.concat(dfs, ignore_index=True)


df = get_all_results()

In [ ]:
# Derive metrics

def fix(row):
    try:
        return int(row.rate.removesuffix("bps"))
    except:
        return None


df["rate_bps"] = df.apply(fix, axis=1)

df['rate_gbps'] = df['rate_bps'] / 1000**3

df['actual_duration_s'] = df['actual_duration_ms'] / 1000

df['pps_out'] = df['stats.total.opackets'] / df['actual_duration_s']
df['mpps_out'] = df['pps_out'] / 1000000

df['pps'] = df['stats.total.ipackets'] / df['actual_duration_s']
df['mpps'] = df['pps'] / 1000000

df['actual_rate_bps_out'] = (df['stats.total.obytes'] * 8) / df['actual_duration_s']
df['actual_rate_gbps_out'] = df['actual_rate_bps_out'] / 1000**3

df['actual_rate_bps'] = (df['stats.total.ibytes'] * 8) / df['actual_duration_s']
df['actual_rate_gbps'] = df['actual_rate_bps'] / 1000**3

df['gap'] = df['actual_rate_bps_out'] / df['rate_bps'] * 100

In [ ]:
for run in df['tag/timestamp'].unique():
    for size in df['size'].unique():
        (so.Plot(df[(df['tag/timestamp'] == run) &  (df['size'] == size)].melt(id_vars= ['iteration'], value_vars=['actual_rate_gbps', 'actual_rate_gbps_out', 'rate_gbps'], var_name='in_out', value_name='y'), x='iteration', y ='y', color='in_out')
         .add(so.Line())
         .label(title=f"{run} {size} bytes")
        ).show()

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='actual_rate_gbps', color='tag/timestamp').add(so.Line())

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='actual_rate_gbps_out', color='tag/timestamp').add(so.Line())

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='mpps', color='tag/timestamp').add(so.Line())

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='mpps_out', color='tag/timestamp').add(so.Line())

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='rate_gbps', color='tag/timestamp').add(so.Line())

In [ ]:
so.Plot(df.loc[df.groupby(["tag", 'timestamp', 'size'])['iteration'].idxmax()], x='size', y='stats.latency.0.latency.average', ymin='stats.latency.0.latency.total_min', ymax='stats.latency.0.latency.l', color='tag/timestamp').add(so.Line()).add(so.Band())